In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
import matplotlib.pyplot as plt

from lavian_et_al_2025.imaging.imaging_classes import LightsheetExperiment
from lavian_et_al_2025.visual_motion.stimulus_functions import stim_vel_dir_dataframe, quantize_directions, make_sensory_regressors, JCh_to_RGB255, get_tuning_map_rois
from lavian_et_al_2025.visual_motion.colors import JCh_to_RGB255

import json
from pathlib import Path

In [ ]:
def color_stack(
        amp,
        angle,
        hueshift=2.5,
        amp_percentile=80,
        maxsat=50,
        lightness_min=100,
        lightness_delta=-40,
    ):
    output_lch = np.zeros((amp.shape[0], 3))
    output_lch[:,0]
    maxamp = np.percentile(amp, amp_percentile)

    output_lch[:, 0] = (
            lightness_min + (np.clip(amp / maxamp, 0, 1)) * lightness_delta
    )
    output_lch[:, 1] = (np.clip(amp / maxamp, 0, 1)) * maxsat
    output_lch[:, 2] = (-angle + hueshift) * 180 / np.pi

    return JCh_to_RGB255(output_lch)

In [ ]:
#Enter path to the dataset
master = Path(r'')
fish_list = list(master.glob("*_f*"))

# Load also morphed coordinates

In [ ]:
morphed_coords = {}
in_brain_arr = {}

for fish in fish_list:
    print(fish)
    
    #Load morphed coords
    morphed_coords[fish.name] = fl.load(fish / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_coords_transformed.h5')
    
    #an index to keep track of ROIs inside brain
    suite2p_brain = fl.load(fish / "data_from_suite2p_cells_brain.h5")
    
    in_brain_arr[fish.name] = np.full(morphed_coords[fish.name].shape[0], False)
    in_brain_arr[fish.name][suite2p_brain['coords_idx']] = True
    
print('Done')

In [ ]:
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

# Morphed datasets

In [ ]:
try:
    tuning_arrs = fl.load(master / 'tuning_arrs.h5')
    amp_pooled, angle_pooled = tuning_arrs['amp_pooled'], tuning_arrs['angle_pooled']

except OSError:
    amp_pooled = {}
    angle_pooled = {}

    for fish in fish_list:
        print(fish)

        #Load traces and stimulus metadata
        exp = glob(str(fish / "*behavior*"))[0]
        traces = fl.load(fish / "filtered_traces.h5", "/detr")

        img_exp = LightsheetExperiment(fish)
        fs = img_exp.fn
        sampling = 1/fs
        time = np.linspace(0, traces.shape[0]*sampling, traces.shape[0])

        #Make list of sensory regressors 
        reg = make_sensory_regressors(Experiment(fish), sampling=sampling)
        reg_list = [reg]

        #Compute tuning
        amp_pooled[fish.name], angle_pooled[fish.name] = get_tuning_map_rois(traces, reg)
        
    tuning_arrs = {'angle_pooled':angle_pooled, 'amp_pooled':amp_pooled}
    fl.save(master / 'tuning_arrs.h5', tuning_arrs)

    print('Done')

In [ ]:
#Pool amp and angle arrays
amp_pooled_arr = np.concatenate([amp_pooled[fish.name] for fish in fish_list])
angle_pooled_arr = np.concatenate([angle_pooled[fish.name] for fish in fish_list])

#also boolean array to keep track of ROIs in the brain
in_brain_arr_pooled = np.concatenate([in_brain_arr[fish.name] for fish in fish_list])

#and pooled coordinates
coords_pooled = np.concatenate([morphed_coords[fish.name] for fish in fish_list], 0)

In [ ]:
#Make filter to exclude NaNs from amp array (we assume NaNs co-occur in amp and angle array, it seems to be the case)
nan_filt = np.isnan(amp_pooled_arr)
print(nan_filt.sum(), ' ROIs excluded')

#Combine into a filtering array
valid_rois = np.logical_and(in_brain_arr_pooled, ~nan_filt)

In [ ]:
#Color stack
colors_ib = color_stack(amp_pooled_arr[valid_rois], angle_pooled_arr[valid_rois])
colors_ib.shape

In [ ]:
#Filter ROIs and normalize vector amplitude
coords_ib_pooled = coords_pooled[valid_rois]
amp_pooled_ib = amp_pooled_arr[valid_rois]
amp_norm = amp_pooled_ib / np.nanmax(amp_pooled_ib)

In [ ]:
#Load reliability arrays and filter ROIs

rel_arr_pooled = np.concatenate([fl.load(fish / "reliability_index_arr.h5", "/reliability_arr_combined") for fish in fish_list])
pct = 98
rel_thresh = np.nanpercentile(rel_arr_pooled, pct) 
print(rel_thresh)
selected_vis = np.where(rel_arr_pooled[~nan_filt[in_brain_arr_pooled]] > rel_thresh)[0]

In [ ]:
coords_vis_pooled = coords_ib_pooled[selected_vis]
colors_vis_pooled = colors_ib[selected_vis]
amp_vis = amp_pooled_ib[selected_vis]

mp_ind_pooled = np.argsort(amp_vis)

# Make figure

In [ ]:
x_lim = [500, 0]
t_lim = [0, 550]
dot_s = 2

scale_bar_len = 100
scale_bar_xpos = 100
scale_bar_ypos1 = 630
scale_bar_ypos2 = 770
fs = 8

In [ ]:
axis_lims = [(l_lim, h_lim) for l_lim, h_lim in zip(np.min(coords_ib_pooled, 0), np.max(coords_ib_pooled, 0))]
axis_lims

In [ ]:
axis_sizes = [lim[1]-lim[0] for lim in axis_lims]
axis_sizes

In [ ]:
min_size = min(axis_sizes)
axis_ratios = [size/min_size for size in axis_sizes]
axis_ratios

In [ ]:
x = 0
y = 1
z = 2

In [ ]:
#Import reference anatomy [from first fish]
ref_anatomy = fl.load(fish_list[0] / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'ref_mapped.h5')

In [ ]:
ref_anatomy_cropped = ref_anatomy[50:550, 70:690, 50:]

In [ ]:
coords_vis_pooled = coords_ib_pooled[selected_vis]
colors_vis_pooled = colors_ib[selected_vis]
amp_vis = amp_pooled_ib[selected_vis]

mp_ind_pooled = np.argsort(amp_vis)

In [ ]:
#Plot
y_main_rat = ref_anatomy_cropped.shape[1]/ref_anatomy_cropped.shape[0]
x_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[0]
y_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[1]*y_main_rat

fig_pool = plt.figure(figsize=(3.5,4))
gs = fig_pool.add_gridspec(2, 2, width_ratios=[1, x_rat], height_ratios=[y_rat, y_main_rat])

ax_pool1 = fig_pool.add_subplot(gs[1, 0])
ax_pool2 = fig_pool.add_subplot(gs[0, 0], sharex=ax_pool1)
ax_pool3 = fig_pool.add_subplot(gs[1, 1], sharey=ax_pool1)

mock_anatomy = np.zeros_like(ref_anatomy_cropped)
ax_pool1.imshow(mock_anatomy.mean(2).T, cmap='gray_r')
ax_pool2.imshow(mock_anatomy.mean(1).T, cmap='gray_r')
ax_pool3.imshow(mock_anatomy.mean(0), cmap='gray_r')

ax_pool1.scatter(coords_ib_pooled[:,0], coords_ib_pooled[:,1]-50, s=dot_s, alpha=1, c='linen')
ax_pool1.scatter(coords_vis_pooled[mp_ind_pooled,0], coords_vis_pooled[mp_ind_pooled,1]-50, c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

ax_pool3.scatter(coords_ib_pooled[:,2], coords_ib_pooled[:,1]-50, s=dot_s, alpha=1, c='linen')
ax_pool3.scatter(coords_vis_pooled[mp_ind_pooled,2], coords_vis_pooled[mp_ind_pooled,1]-50, c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

ax_pool2.scatter(coords_ib_pooled[:,0], coords_ib_pooled[:,2]+30, s=dot_s, alpha=1, c='linen')
ax_pool2.scatter(coords_vis_pooled[mp_ind_pooled,0], coords_vis_pooled[mp_ind_pooled,2]+30, c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

#Scale bars
scale_bar_ypos1 = 720
ax_pool1.plot((scale_bar_xpos, scale_bar_xpos+scale_bar_len), (scale_bar_ypos1, scale_bar_ypos1), c='black')
ax_pool1.text(scale_bar_xpos+(scale_bar_len/2), scale_bar_ypos1+10, r'{}$\mu$m'.format(scale_bar_len), va='top', ha='center', fontsize=fs)

ax_pool2.invert_yaxis()

for ax in [ax_pool1, ax_pool2, ax_pool3]:
     ax.axis('off')
        
plt.subplots_adjust(hspace=0.01, wspace=0.00, left=0.05, right=0.95, top=0.95, bottom=0.05)